In [ ]:
# ISP Customer Care Chat

A full Gradio chat interface for an Internet Service Provider that:
- **Placates angry customers** (outages, complaints)
- **Uses a tool** when customers complain about **speed** → fetches a higher-tier package and suggests an upsell
- **Model switching** via **OpenRouter** — Claude and OpenAI models only

In [24]:
# Imports
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [25]:
# Environment and API clients — OpenRouter only (Claude + OpenAI models)
load_dotenv(override=True)
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
openrouter_base_url = "https://openrouter.ai/api/v1"

if openrouter_api_key:
    print(f"OpenRouter API key loaded (starts with {openrouter_api_key[:8]}...)")
else:
    print("OPENROUTER_API_KEY not set — add it to .env")

# OpenRouter client for Claude and OpenAI models
openrouter_client = (
    OpenAI(base_url=openrouter_base_url, api_key=openrouter_api_key)
    if openrouter_api_key else None
)

OpenRouter API key loaded (starts with sk-or-v1...)


In [39]:
# Model config: (display_name, client_key, model_id) — Claude and OpenAI via OpenRouter only
# OpenRouter model IDs: https://openrouter.ai/docs#models
MODEL_CONFIG = [
    ("GPT-4o Mini (OpenRouter)", "openrouter", "openai/gpt-4o-mini"),
    ("GPT-4o (OpenRouter)", "openrouter", "openai/gpt-4o"),
    ("Claude 3.5 Sonnet (OpenRouter)", "openrouter", "anthropic/claude-3.5-sonnet"),
]

ISP_SYSTEM_PROMPT = """You are a friendly, empathetic customer service agent for a residential Internet Service Provider (ISP) that sometimes experiences outages.

Your goals:
1. **Placate angry customers**: Acknowledge their frustration, apologize for outages or poor experience, and show you're here to help. Be calm and professional.
2. **Outages**: If they report an outage, apologize, say you've logged it and that the team is working on it. Offer to check their account or send updates if they provide details.
3. **Speed complaints (IMPORTANT)**: When a customer mentions slow internet, speed, connection quality, or wants a better plan, you MUST call the tool `get_higher_tier_package` first. Do not reply about plans or prices without calling this tool. After you receive the tool result, use the exact plan name, price_monthly, and benefits from the result in your reply. Tell the customer the plan name and the monthly price (e.g. "Fiber Pro 500 for $79.99/month") and 1–2 benefits. Never invent prices or plan names—only use data returned by the tool.
4. Keep responses concise (2–4 sentences unless explaining a plan). Be warm but professional. Never be defensive or argue with the customer."""

In [38]:
# Tool: get details of a higher-tier internet package (for upsell when customer complains about speed)
def get_higher_tier_package():
    """Return details of the next tier up for upsell. Call this when the customer complains about speed or slow internet."""
    return {
        "name": "Fiber Pro 500",
        "speed_down_mbps": 500,
        "speed_up_mbps": 50,
        "price_monthly": 79.99,
        "benefits": [
            "Up to 500 Mbps download / 50 Mbps upload",
            "Fiber to the home where available",
            "Priority support and fewer slowdowns during peak hours",
            "Free modem upgrade",
        ],
        "typical_current_tier": "Basic 100 (100 Mbps)",
    }

# day4-style: function dict then wrap in tools list
get_higher_tier_package_function = {
    "name": "get_higher_tier_package",
    "description": "Get the details of a higher-tier internet package (name, speed, price, benefits). Call this when the customer complains about slow internet, speed, or connection quality so you can suggest an upgrade.",
    "parameters": {
        "type": "object",
        "properties": {},
        "required": [],
        "additionalProperties": False,
    },
}
tools = [{"type": "function", "function": get_higher_tier_package_function}]

In [40]:
def get_client(model_choice):
    """Resolve client and model id from dropdown choice (OpenRouter only)."""
    choice = model_choice or MODEL_CONFIG[0][0]
    for display_name, client_key, model_id in MODEL_CONFIG:
        if display_name == choice:
            return openrouter_client, model_id
    return openrouter_client, MODEL_CONFIG[0][2]


def handle_tool_calls(message):
    """Execute tool calls from the assistant message (day4 format)."""
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_higher_tier_package":
            arguments = json.loads(tool_call.function.arguments or "{}")
            package = get_higher_tier_package()
            # day4 style: content is a string for the model to read
            price_str = f"${package['price_monthly']}/month"
            benefits_str = "; ".join(package["benefits"][:2])
            content = f"The higher-tier plan is {package['name']} at {price_str} ({package['speed_down_mbps']} Mbps down / {package['speed_up_mbps']} Mbps up). Benefits: {benefits_str}."
            responses.append({
                "role": "tool",
                "content": content,
                "tool_call_id": tool_call.id,
            })
    return responses

In [42]:
def _content_to_string(content):
    """Normalize message content to a string (Gradio may return list/dict after first turn)."""
    if content is None:
        return ""
    if isinstance(content, str):
        return content
    if isinstance(content, list):
        parts = []
        for block in content:
            if isinstance(block, dict) and "text" in block:
                parts.append(block["text"])
            elif isinstance(block, str):
                parts.append(block)
        return " ".join(parts) if parts else ""
    if isinstance(content, dict) and "text" in content:
        return content["text"]
    return str(content)


def chat(message, history, model_choice):
    """Chat with the ISP assistant. Supports tool calls (e.g. get_higher_tier_package) and model switching."""
    if not message or not message.strip():
        return history

    client, model_id = get_client(model_choice)
    if client is None:
        return history + [
            {"role": "user", "content": message},
            {"role": "assistant", "content": "No API client available. Set OPENROUTER_API_KEY in .env or run Ollama."},
        ]

    messages = [{"role": "system", "content": ISP_SYSTEM_PROMPT}]
    for h in history:
        role = h.get("role") or "user"
        content = _content_to_string(h.get("content"))
        messages.append({"role": role, "content": content})
    messages.append({"role": "user", "content": message.strip()})

    # day4 format: tools=tools, no tool_choice; on tool_calls append message object and extend responses
    use_tools = openrouter_client and client == openrouter_client
    kwargs = {"model": model_id, "messages": messages}
    if use_tools:
        kwargs["tools"] = tools

    response = client.chat.completions.create(**kwargs)
    msg = response.choices[0].message

    while response.choices[0].finish_reason == "tool_calls":
        message_obj = response.choices[0].message
        responses = handle_tool_calls(message_obj)
        messages.append(message_obj)
        messages.extend(responses)
        if use_tools:
            response = client.chat.completions.create(model=model_id, messages=messages, tools=tools)
        else:
            response = client.chat.completions.create(model=model_id, messages=messages)
    msg = response.choices[0].message

    assistant_content = (msg.content or "").strip() or "I’m sorry, I couldn’t generate a response."
    return history + [
        {"role": "user", "content": message},
        {"role": "assistant", "content": assistant_content},
    ]

In [43]:
# Gradio UI: chat + model selector — use gr.State() for history so subsequent messages work
def respond(message, history, model_choice):
    """Chat with the assistant. History comes from State, not Chatbot."""
    if not message or not message.strip():
        yield history, "", history
        return
    new_history = chat(message, history, model_choice)
    yield new_history, "", new_history


with gr.Blocks(title="ISP Customer Care", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🌐 ISP Customer Care\n*Placate upset customers, handle outages, and upsell on speed.*")
    gr.Markdown("Try: *\"My internet is so slow!\"* or *\"There's been an outage for hours!\"*")

    model_dropdown = gr.Dropdown(
        choices=[m[0] for m in MODEL_CONFIG],
        value=MODEL_CONFIG[0][0],
        label="Model",
    )
    chatbot = gr.Chatbot(type="messages", height=400, label="Chat")
    chat_history_state = gr.State([])  # source of truth for history; Chatbot is display only
    msg = gr.Textbox(placeholder="Type your message...", label="Message", scale=4)
    submit_btn = gr.Button("Send", variant="primary", scale=1)

    submit_btn.click(
        fn=respond,
        inputs=[msg, chat_history_state, model_dropdown],
        outputs=[chatbot, msg, chat_history_state],
    )
    msg.submit(
        fn=respond,
        inputs=[msg, chat_history_state, model_dropdown],
        outputs=[chatbot, msg, chat_history_state],
    )

demo.launch()

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.
